In [1]:
from tqdm import tqdm_notebook as tqdm
import os
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
!apt install libspatialindex-dev
!pip install rtree
!pip install geopandas
import geopandas as gpd
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from google.colab import drive
drive.mount('/content/drive')

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libspatialindex-c6 libspatialindex6
The following NEW packages will be installed:
  libspatialindex-c6 libspatialindex-dev libspatialindex6
0 upgraded, 3 newly installed, 0 to remove and 31 not upgraded.
Need to get 319 kB of archives.
After this operation, 1,416 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libspatialindex6 amd64 1.9.3-2 [247 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libspatialindex-c6 amd64 1.9.3-2 [55.8 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libspatialindex-dev amd64 1.9.3-2 [16.0 kB]
Fetched 319 kB in 0s (1,269 kB/s)
Selecting previously unselected package libspatialindex6:amd64.
(Reading database ... 121730 files and directories currently installed.)
Preparing to unpack .../libspatialindex6_1.9.3-2_amd64.deb ...
U

In [2]:
# 读取 shapefile 文件
shapefile_path = "/content/drive/MyDrive/SD/powerplant.shp"
gdf = gpd.read_file(shapefile_path)

# 将 GeoDataFrame 转换为 Pandas DataFrame
powerplants = gdf.drop(columns='geometry')

# 获取 P03_0005 列的前四个字符并转换为整数
powerplants.rename(columns = {'P03_0001': 'Company','P03_0003':'Address','P03_0005':'Update_Year','P03_0006':'Max_Output'}, inplace = True)
powerplants = powerplants[['Company','Address','Update_Year','Max_Output','Prefecture']]
powerplants['Update_Year'] = powerplants['Update_Year'].apply(lambda x: int(str(x)[:4]))
powerplants['Company'] = powerplants['Company'].apply(lambda x: str(x)[:-4])

powerplants

,Company,Address,Update_Year,Max_Output,Prefecture
0,沖縄電力,沖縄県うるま市字宇堅657番地,1994,156.0,okinawa
1,沖縄電力,沖縄県うるま市字宇堅657番地,1995,156.0,okinawa
2,九州電力,熊本県天草郡苓北町年柄字苓陽1091番地,1995,700.0,kumamoto
3,九州電力,熊本県天草郡苓北町年柄字苓陽1091番地,2003,700.0,kumamoto
4,九州電力,長崎県松浦市志佐町白浜免字開発2091番地1,1989,700.0,nagasaki
...,...,...,...,...,...
283,東北電力,新潟県北蒲原郡聖籠町東港1丁目1-155,2011,53.8,niigata
284,東京電力,神奈川県横須賀市久里浜9-2-1,2011,75.9,kanagawa
285,東京電力,神奈川県横須賀市久里浜9-2-1,2011,102.1,kanagawa
286,東京電力,神奈川県横須賀市久里浜9-2-1,2011,151.6,kanagawa


In [3]:
Intensity = pd.read_excel('/content/drive/MyDrive/SD/Intensity.xlsx')
Intensity

,Fuel,Year,Heat,Carbon
0,Coal,1990,26.0,23.65
1,Coal,2005,25.7,24.51
2,Coal,2007,25.7,24.51
3,Coal,2008,25.7,24.51
4,Coal,2009,25.7,24.51
...,...,...,...,...
59,Gas,2016,54.5,13.95
60,Gas,2017,54.5,13.95
61,Gas,2018,54.7,13.87
62,Gas,2019,54.7,13.87


In [4]:
pg = pd.read_excel('/content/drive/MyDrive/SD/power_generation.xlsx').iloc[1:].fillna(0)
pg['年期'] = pg['年期'].apply(lambda x: int(x))
pg[pg.columns[2:]] = pg[pg.columns[2:]].astype('float')
pg.rename(columns = {'会社名': 'Company','年期':'Year','発電':'Power','石炭湿_湿':'Coal','ＬＮＧ_発電用':'Gas','ＬＰＧ_発電用':'LPG'}, inplace = True)
pg['Oil'] = pg['重油ブレンド分調整前_発電用'] + pg['原油ブレンド分調整前_発電用']
pg = pg[['Company','Year','Power','Coal','Oil','Gas','LPG']]
pg[pg.columns[3:]] = pg[pg.columns[3:]] / 1000
pg

,Company,Year,Power,Coal,Oil,Gas,LPG
1,北海道電力,2015,22068237.0,5521.852,1696.319,0.000,0.000
2,東北電力,2015,56930988.0,8290.920,696.816,4320.034,0.000
3,東京電力,2015,198015773.0,8547.828,2495.190,21549.707,156.368
4,中部電力,2015,111219523.0,10168.939,61.645,11725.185,0.000
5,北陸電力,2015,22329227.0,6667.041,572.192,0.000,0.000
...,...,...,...,...,...,...,...
526,関西電力,1963,16698869.0,4375.605,1809.774,0.000,0.000
527,中国電力,1963,3459881.0,1595.019,238.428,0.000,0.000
528,四国電力,1963,1360612.0,390.332,167.868,0.000,0.000
529,九州電力,1963,6721434.0,3153.377,158.868,0.000,0.000


In [23]:
year_list = [1990,2005,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020]

for year in year_list:
  if year > 2015:
    grouped_df['Year'] = year
    eheat = eheat.append(grouped_df)
  else:
    # 选择 'Year' 列值小于等于 year 的行
    filtered_rows = powerplants[powerplants['Update_Year'] <= year]

    # 对 'Address' 列进行分组，选择每组中 'Update_Year' 列最大的一行
    result = filtered_rows.loc[filtered_rows.groupby('Address')['Update_Year'].idxmax()]

    # 1. 按照 "Company" 和 "Prefecture" 两列分组，并对 "Max_Output" 列应用 sum 运算
    grouped_df = result.groupby(['Company', 'Prefecture'])['Max_Output'].sum().reset_index()

    # 2. 计算每个 Prefecture 的 Max_Output 占单个 Company 的总 Max_Output 的比例
    grouped_df['Ratio'] = grouped_df.groupby('Company')['Max_Output'].transform(lambda x: x / x.sum())

    # 1. 从两个数据框中筛选出 "Year" 列等于 year 的行
    filtered_pg = pg[pg['Year'] == year]
    filtered_intensity = Intensity[Intensity['Year'] == year]

    # 2. 在 filtered_pg 数据框中进行计算
    for fuel in ['Coal', 'Oil', 'Gas', 'LPG']:
        heat, carbon = filtered_intensity.loc[filtered_intensity['Fuel'] == fuel, ['Heat', 'Carbon']].values[0]
        filtered_pg[fuel] = filtered_pg[fuel] * heat

    # 3. 计算 Coal、Oil、Gas、LPG 四列的总和，并新增一列 "sum"
    filtered_pg['sum'] = filtered_pg[['Coal', 'Oil', 'Gas', 'LPG']].sum(axis=1)

    # 4. 在 grouped_df 中新增一列 "Electricity" 计算每个 Prefecture 的碳排放
    grouped_df['Electricity'] = grouped_df['Ratio'] * grouped_df['Company'].map(filtered_pg.groupby('Company')['sum'].sum())
    grouped_df['Year'] = year

    # 储存数据
    if year == 1990:
      eheat = grouped_df
    else:
      eheat = eheat.append(grouped_df)

eheat

,Company,Prefecture,Max_Output,Ratio,Electricity,Year
0,中国電力,okayama,656.0,0.238026,47128.402200,1990
1,中国電力,yamaguchi,2100.0,0.761974,150868.360700,1990
2,中部電力,aichi,2850.0,0.614887,439045.548731,1990
3,中部電力,mie,1785.0,0.385113,274981.159469,1990
4,九州電力,fukuoka,1475.0,0.375480,88763.935448,1990
...,...,...,...,...,...,...
28,沖縄電力,okinawa,1093.0,1.000000,62053.934000,2020
29,関西電力,hyougo,2174.5,0.347948,240447.247803,2020
30,関西電力,kyouto,1275.0,0.204016,140984.245090,2020
31,関西電力,osaka,1600.0,0.256020,176921.405604,2020


In [24]:
eheat.to_excel('/content/drive/MyDrive/SD/Electricity_heat.xlsx', index=False)

In [25]:
year_list = [1990,2005,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020]

for year in year_list:
  if year > 2015:
    grouped_df['Year'] = year
    emission = emission.append(grouped_df)
  else:
    # 选择 'Year' 列值小于等于 year 的行
    filtered_rows = powerplants[powerplants['Update_Year'] <= year]

    # 对 'Address' 列进行分组，选择每组中 'Update_Year' 列最大的一行
    result = filtered_rows.loc[filtered_rows.groupby('Address')['Update_Year'].idxmax()]

    # 1. 按照 "Company" 和 "Prefecture" 两列分组，并对 "Max_Output" 列应用 sum 运算
    grouped_df = result.groupby(['Company', 'Prefecture'])['Max_Output'].sum().reset_index()

    # 2. 计算每个 Prefecture 的 Max_Output 占单个 Company 的总 Max_Output 的比例
    grouped_df['Ratio'] = grouped_df.groupby('Company')['Max_Output'].transform(lambda x: x / x.sum())

    # 1. 从两个数据框中筛选出 "Year" 列等于 year 的行
    filtered_pg = pg[pg['Year'] == year]
    filtered_intensity = Intensity[Intensity['Year'] == year]

    # 2. 在 filtered_pg 数据框中进行计算
    for fuel in ['Coal', 'Oil', 'Gas', 'LPG']:
        heat, carbon = filtered_intensity.loc[filtered_intensity['Fuel'] == fuel, ['Heat', 'Carbon']].values[0]
        filtered_pg[fuel] = filtered_pg[fuel] * heat * carbon / 1000

    # 3. 计算 Coal、Oil、Gas、LPG 四列的总和，并新增一列 "sum"
    filtered_pg['sum'] = filtered_pg[['Coal', 'Oil', 'Gas', 'LPG']].sum(axis=1)

    # 4. 在 grouped_df 中新增一列 "Electricity" 计算每个 Prefecture 的碳排放
    grouped_df['Electricity'] = grouped_df['Ratio'] * grouped_df['Company'].map(filtered_pg.groupby('Company')['sum'].sum()) * 44 / 12
    grouped_df['Year'] = year

    # 储存数据
    if year == 1990:
      emission = grouped_df
    else:
      emission = emission.append(grouped_df)

emission

,Company,Prefecture,Max_Output,Ratio,Electricity,Year
0,中国電力,okayama,656.0,0.238026,3555.061967,1990
1,中国電力,yamaguchi,2100.0,0.761974,11380.533735,1990
2,中部電力,aichi,2850.0,0.614887,26502.166070,1990
3,中部電力,mie,1785.0,0.385113,16598.725065,1990
4,九州電力,fukuoka,1475.0,0.375480,5844.847297,1990
...,...,...,...,...,...,...
28,沖縄電力,okinawa,1093.0,1.000000,4994.373069,2020
29,関西電力,hyougo,2174.5,0.347948,14574.759748,2020
30,関西電力,kyouto,1275.0,0.204016,8545.789229,2020
31,関西電力,osaka,1600.0,0.256020,10724.127660,2020


In [21]:
emission.groupby('Year')['Electricity'].sum()

Year
1990    217149.756166
2005    256036.821960
2007    297190.987608
2008    279820.388772
2009    240291.804717
2010    262929.104843
2011    320318.653449
2012    348289.922795
2013    363326.941530
2014    339448.454656
2015    319451.053585
Name: Electricity, dtype: float64

In [26]:
emission.to_excel('/content/drive/MyDrive/SD/Electricity_emission.xlsx', index=False)

In [11]:
# 定义 year
year = 1990

# 选择 'Year' 列值小于等于 year 的行
filtered_rows = powerplants[powerplants['Update_Year'] <= year]

# 对 'Address' 列进行分组，选择每组中 'Update_Year' 列最大的一行
result = filtered_rows.loc[filtered_rows.groupby('Address')['Update_Year'].idxmax()]

# 打印结果
result

,Company,Address,Update_Year,Max_Output,Prefecture
51,中部電力,三重県三重郡川越町大字亀崎新田字朝明87-1,1990,700.0,mie
49,中部電力,三重県四日市市三郎町1,1988,585.0,mie
124,中部電力,三重県尾鷲市国市松泉町1,1987,500.0,mie
140,関西電力,京都府宮津市小田宿野1001,1989,375.0,kyouto
99,九州電力,佐賀県唐津市二夕子3丁目213番地,1973,500.0,saga
...,...,...,...,...,...
175,四国電力,香川県坂出市番の州町2,1974,350.0,kagawa
163,九州電力,鹿児島県大島郡与論町立長296−2,1976,1.1,kagoshima
165,九州電力,鹿児島県大島郡喜界町塩道19−1,1977,1.1,kagoshima
164,九州電力,鹿児島県大島郡知名町知名1873,1976,1.1,kagoshima


In [12]:
# 假设 result_df 是您上一个操作得到的结果

# 1. 按照 "Company" 和 "Prefecture" 两列分组，并对 "Max_Output" 列应用 sum 运算
grouped_df = result.groupby(['Company', 'Prefecture'])['Max_Output'].sum().reset_index()

# 2. 计算每个 Prefecture 的 Max_Output 占单个 Company 的总 Max_Output 的比例
grouped_df['Ratio'] = grouped_df.groupby('Company')['Max_Output'].transform(lambda x: x / x.sum())

# 打印结果
grouped_df

,Company,Prefecture,Max_Output,Ratio
0,中国電力,okayama,656.0,0.238026
1,中国電力,yamaguchi,2100.0,0.761974
2,中部電力,aichi,2850.0,0.614887
3,中部電力,mie,1785.0,0.385113
4,九州電力,fukuoka,1475.0,0.375480
5,九州電力,kagoshima,503.3,0.128122
6,九州電力,nagasaki,1200.0,0.305476
7,九州電力,ooita,250.0,0.063641
8,九州電力,saga,500.0,0.127282
9,北海道電力,hokkaidou,1924.0,1.000000


In [15]:
import pandas as pd

# 假设 pg 和 Intensity 是您的两个数据框
# 假设 year 已经被定义为 1990

# 1. 从两个数据框中筛选出 "Year" 列等于 year 的行
filtered_pg = pg[pg['Year'] == year]
filtered_intensity = Intensity[Intensity['Year'] == year]

# 2. 在 filtered_pg 数据框中进行计算
for fuel in ['Coal', 'Oil', 'Gas', 'LPG']:
    heat, carbon = filtered_intensity.loc[filtered_intensity['Fuel'] == fuel, ['Heat', 'Carbon']].values[0]
    filtered_pg[fuel] = filtered_pg[fuel] * heat * carbon / 1000

# 3. 计算 Coal、Oil、Gas、LPG 四列的总和，并新增一列 "sum"
filtered_pg['sum'] = filtered_pg[['Coal', 'Oil', 'Gas', 'LPG']].sum(axis=1)

# 4. 在 grouped_df 中新增一列 "Electricity" 计算每个 Prefecture 的碳排放
grouped_df['Electricity'] = grouped_df['Ratio'] * grouped_df['Company'].map(filtered_pg.groupby('Company')['sum'].sum())*44/12000

# 打印结果
grouped_df


,Company,Prefecture,Max_Output,Ratio,Electricity
0,中国電力,okayama,656.0,0.238026,3.555062
1,中国電力,yamaguchi,2100.0,0.761974,11.380534
2,中部電力,aichi,2850.0,0.614887,26.502166
3,中部電力,mie,1785.0,0.385113,16.598725
4,九州電力,fukuoka,1475.0,0.375480,5.844847
5,九州電力,kagoshima,503.3,0.128122,1.994381
6,九州電力,nagasaki,1200.0,0.305476,4.755130
7,九州電力,ooita,250.0,0.063641,0.990652
8,九州電力,saga,500.0,0.127282,1.981304
9,北海道電力,hokkaidou,1924.0,1.000000,11.135435


In [16]:
grouped_df['Electricity'].sum()

217.14975616637597